# Apartments Analysis

In this project, we will analyze Apartments for rent in the city of Tel Aviv.        
Our main goals are to try to understand in depth the area of apartment rental in Tel Aviv.   

## Introduction and Background
Finding an apartment for rent is not a simple task. And it is usually necessary to see a large number of apartments 
and be prepared for certain compromises before we can make a decision and enter any apartment.  
For that, we performed in-depth analysis to understand the field in depth.   
Our motivation in choosing a project topic in this area comes from the fact that we find ourselves  
spending a lot of time, each day, for finding apartments for rent. 

## Data acquisition
For the purposes of this project, we will use data collected from ads which have been published on the site Yad2.   
All the information has been collected using a Web crawler.    


### Work through the notebook
First we will import all the relevant dependencies we need.

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

#### Loading in the data  
Our data is stored and organized in a csv file.  
We'll start reading in the data into pandas dataframe and see some information.  

In [ ]:
# Read in the data
apartments = pd.read_csv('apartments.csv')

print(f'Apartments dataset has {apartments.shape[0]} entries with {apartments.shape[1]} variables.')

In [ ]:
apartments.head()

### Feature Observation

In [ ]:
# Data features
apartments[0:0]

- **Address**: the actual apartment address
- **Area**: we divided the city into areas north-south-east-west
- **Area-text**: the neighborhood where the apartment is located
- **Price**: price of rent per month in ILS
- **Rooms**: number of rooms in the apartment
- **Floor**: number of floor of the apartment in the building
- **Square-meter**: the size of the apartment space in square meters
- **Elevator/air-condition/refurbished/furniture**: additional parameters that can be exist or not
- **lon, lat**: Longitude and Latitude coordinates

## Data cleaning

In [ ]:
apartments.info()

### Handling missing data and duplicates
Some of the values are missing, they anotated as 'לא צוין'  
We will also delete the duplicated data

In [ ]:
# Remove missing values
apartments = apartments[apartments['Price'].str.strip() != 'לא צוין מחיר']
apartments = apartments[apartments['Rooms'].str.strip() != 'לא צוין']
apartments = apartments[apartments['Square-meter'].str.strip() != 'לא צוין']

# Drop duplicated data
apartments.drop_duplicates(inplace=True)

### Handling Data types
Lets look at the variables type

In [ ]:
apartments.dtypes

We can see that some of the variables are not in the correct type, such as **Price**, **Rooms**, **Floor** so we will need to convert them.

In [ ]:
# Fix values
apartments['Price'] = apartments['Price'].apply(lambda s: s.replace(',', '').replace('$', '').replace('₪', '')).astype('float')
apartments.loc[apartments['Floor'] == 'קרקע', 'Floor'] = 0

# Convert data types into a valid type
apartments['Price'] = apartments['Price'].astype('float')
apartments['Rooms'] = apartments['Rooms'].astype('float')
apartments['Floor'] = apartments['Floor'].astype('int')
apartments['Square-meter'] = apartments['Square-meter'].astype('int')

apartments.dtypes

### Remove Outliers


In [ ]:
sns.boxplot(apartments['Price'])

Looks like we have some outliers

In [ ]:
# Remove outliers
apartments = apartments[apartments['Price'] < 30000]

Some of the data contains parking and storage for rent, we will filter them by price and square-meter

In [ ]:
# Filtering parkings/storages
apartments = apartments[apartments['Price'] > 1000]
apartments = apartments[apartments['Square-meter'] > 15]

print(f'Now the dataset has {apartments.shape[0]} entries.')

sns.boxplot(apartments['Price'])

## Let's explore some of the features

#### Elevator, Air-condition, Refurbished, Furniture
How many apartments do they have and some do not?

In [ ]:
# Plot on the same figure
fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(15, 4))

sns.countplot(x='Air-condition', data=apartments, ax=ax1)
sns.countplot(x='Elevator', data=apartments, ax=ax2)
sns.countplot(x='Refurbished', data=apartments, ax=ax3)
sns.countplot(x='Furniture', data=apartments, ax=ax4)

# Fix layout spaces
plt.tight_layout()

Almost all of the apartments have air condition, still .. this is Tel Aviv

### Rooms
The distribution of number of rooms in the apartments

In [ ]:
sns.distplot(apartments['Rooms'])

We can see that the average number of rooms is 3

### Price
The prices are in ILS (israeli shekels)

In [ ]:
sns.distplot(apartments['Price'], color='r')

# Price stats
apartments['Price'].describe().round()

### The relationship between the variables

In [ ]:
sns.pairplot(apartments[['Price', 'Square-meter', 'Rooms', 'Floor', 'Area']], hue='Area')

## Exploring Interesting Data
Let's see some intresting insights about the data

### what is the avereage price per area?

In [ ]:
avg_per_area = apartments.groupby('Area')['Price'].mean()
avg_per_area.plot.bar(rot=40)
avg_per_area.round()

We can see that the **center-lev-tlv** and the **north** are the most expensive areas.   
it is legitimate because Tel Aviv is a young city and most of the entertainment places are in these areas

### The relationship between number of rooms and square-meter

In [ ]:
rounded_rooms = apartments[apartments.Rooms.apply(lambda x: x % 1 == 0)]
sns.scatterplot(x='Square-meter', y='Rooms', hue='Rooms', data=rounded_rooms, palette="Set2")
plt.legend(loc='upper right')

### What is the price for each square meter in each area?

In [ ]:
apartments['newArea'] = apartments['Area']
apartments.loc[apartments['Area'].str.contains('center'), 'newArea'] = 'center'
apartments['price_to_meter'] = apartments['Price'] / apartments['Square-meter']
grouped = apartments.groupby('newArea')['price_to_meter'].mean()
grouped

### The relationship between **Price**  and **Square-meter**

In [ ]:
sns.scatterplot(x='Square-meter', y='Price', data=apartments, hue='Price')

## Geographical View
### how these prices look geographically? 

In [ ]:
import folium

In [ ]:
# Initialialize map
m = folium.Map(location=[32.073333, 34.7999900], zoom_start=14)

avg_per_meter = apartments.groupby('Area-text')['price_to_meter'].mean().to_dict()

# Set colors per apartment
apartments['color'] = apartments.apply(lambda apt: '#FF1700' if (avg_per_meter[apt['Area-text']] < apt['price_to_meter']) else '#2CD31D', axis=1)

for index, row in apartments.iterrows():
    if not np.isnan(row.lat):
        marker = folium.CircleMarker(
            location=[row.lat, row.lon],
            radius=row['Square-meter'] % 10,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            fill_opacity='0.3',
            popup=row.Address)
        m.add_child(marker)
m

**Circle size**: according to the apartment size   
**Red**: high price to meter ratio in neighberhood  
**Green**: low price to meter ratio in neighberhood

## Machine Learning

In [ ]:
# apartments_north = apartments[apartments['Area']=='north'].groupby('Area-text')['Price'].mean().plot.bar(rot=40)
# apartments_north

# apartments_south = apartments[apartments['Area']=='south'].groupby('Area-text')['Price'].mean().plot.bar(rot=40)
# apartments_south

apartments_center = apartments[apartments['newArea']=='center'].groupby('Area-text')['Price'].mean().plot.bar(rot=40)
apartments_center



In [ ]:
# from sklearn import linear_model
# from scipy.stats import logistic
# from scipy.special import expit
# from scipy.special import logit
# from sklearn.neighbors import KNeighborsClassifier
# from matplotlib.colors import ListedColormap
# from sklearn import neighbors, datasets

In [ ]:
# plt.scatter(x=apartments.Rooms,y=apartments.Price,c='r',marker='s',label='Room')
# plt.legend(numpoints=1,loc=4)
# plt.show()
# plt.scatter(x=apartments['Square-meter'],y=apartments.Price,c='g',marker='o',label='Square meter')
# plt.legend(numpoints=1,loc=4)
# plt.show()
# plt.scatter(x=apartments.Floor,y=apartments.Price,c='k',marker='*',label='Floor')
# plt.legend(numpoints=1,loc=4)
# plt.show()

In [ ]:
# def measure(col):
#     md_r = linear_model.LinearRegression()
#     x = apartments[col].values.reshape(apartments[col].shape[0], 1)
#     y = apartments['Price'].values.reshape(apartments['Price'].shape[0], 1)
#     md_r.fit(X=x, y=y)
#     print("Slope:",md_r.coef_)
#     print("Intercept:",md_r.intercept_)
#     print("R2:",  md_r.score(x,y))  

# measure('Square-meter')

In [ ]:
# measure('Rooms')

In [ ]:
# measure('Floor')

In [ ]:
# md_r = linear_model.LinearRegression()
# apartments.Refurbished = apartments.Refurbished.astype(int)
# x = apartments[['Floor','Rooms','Square-meter']].values.reshape(apartments[['Floor','Rooms','Square-meter']].shape[0],3)
# y = apartments['Price'].values.reshape(apartments['Price'].shape[0],1)
# md_r.fit(X=x, y=y)
# print("Slope:",md_r.coef_)
# print("Intercept:",md_r.intercept_)
# print("R2:",md_r.score(x,y))

In [ ]:
# df = apartments[['Price','Refurbished']]
# df.Refurbished = df.Refurbished.astype(int)
# df.plot.scatter(x='Refurbished',y='Price', s=30)

In [ ]:
# df = apartments[['Rooms','Square-meter','Price','Area']]
# X = df[['Square-meter','Price']]
# y = df.Area 
# g = sns.scatterplot(X.iloc[:, 0], X.iloc[:, 1], hue=y,  marker="o" ,s=30)
# g.set(xscale='log', yscale='log')
# x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 1].max() + 1
# y_min, y_max = X.iloc[:, 0].min() - 1, X.iloc[:, 1].max() + 1

In [ ]:
# n_neighbors = 10

# # we create an instance of Neighbours Classifier and fit the data.
# clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
# clf.fit(X, apartments['Area'])
# x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
# y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1

# print(f'x: {x_min} , {x_max}') # x: 17 , 341
# print(f'y: {y_min} , {y_max}') # y: 1299.0 , 70001.0,  Most of the vvalues are beteen 1  - 20,000

# # predict class using data and kNN classifier
# size  = range(x_min,x_max)
# y_min,y_max = 1000,20000
# y_jump = round((y_max-y_min) / len(x))
# print(f'y_jump={y_jump}') # y_jump=40
# price = list(range(y_min,y_max,y_jump))
# print(f'size len = {len(size)}')
# print(f'price len = {len(price)}')
# n=320
# xx, yy = np.meshgrid(size[:n],price[:n])
# i = np.c_[xx.ravel(), yy.ravel()]
# predict_vector = clf.predict(i)
# i, predict_vector

In [ ]:
# plt.hist(predict_vector)

In [ ]:
# print(set(predict_vector))
# area_to_number = {v:index for  index,v in enumerate(list(set(predict_vector)))}
# Z1 = [area_to_number[area] for area in predict_vector]
# print(set(Z1))
# Z = np.asarray(Z1[:n*n]).reshape(n,n)
# Z

In [ ]:
# # Create color maps
# plt.title(f"3-Class classification (k = {n_neighbors})")
# h = plt.pcolormesh(xx,yy,Z)
# plt.colorbar(h)
# plt.show()